In [ ]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from torch import nn

# **Dataset**

In [ ]:
def imshow(img):
    img = img / 2 + 0.5
    img = img.numpy()
    plt.imshow(np.transpose(img, (1,2,0)))
    plt.show()

In [ ]:
transforms = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

In [ ]:
training_data = torchvision.datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=transforms
)
testing_data = torchvision.datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=transforms
)

In [ ]:
class_names = [ "airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
training_dataloader = DataLoader(training_data, batch_size=4, shuffle=False)
Img, Lbl = next(iter(training_dataloader))
imshow(torchvision.utils.make_grid(Img))
print(' '.join(f'{class_names[Lbl[j]]:5s}' for j in range(4)))

# **Model**

In [ ]:
from torch.nn.modules.pooling import MaxPool2d
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.Stage_1 = nn.Sequential(
            nn.Conv2d(3, 6, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.Stage_2 = nn.Flatten()
        self.Stage_3 = nn.Sequential(
            nn.Linear(16*5*5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10)
        )
        

    def forward(self, X):
        Stage_1 = self.Stage_1(X)
        Stage_2 = self.Stage_2(Stage_1)
        Stage_3 = self.Stage_3(Stage_2)
        return Stage_3

    def train(self, training_data, optimizer="sgd", batch_size=64, epochs=10, lr=5e-6):
        training_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=2)
        Loss = nn.CrossEntropyLoss()
        if optimizer.lower()=="adam":
            Optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        else:
            Optimizer = torch.optim.SGD(self.parameters(), lr=lr, momentum=0.9)

        print("-----------------------------------------------")
        for i in range(epochs):
            for batch, (X, y) in enumerate(training_dataloader):
                y_pre = self.forward(X)
                loss_fn = Loss(y_pre, y)

                Optimizer.zero_grad()
                loss_fn.backward()
                Optimizer.step()
            if (i%10 == 0):
                print(f"-- Epoch {i}/{epochs}: Loss = {loss_fn}")

    def evaluate(self, testing_data):
        corrected, total = 0, 0
        testing_dataloader = DataLoader(testing_data, batch_size=1, shuffle=False)

        for batch, (X, y) in enumerate(testing_dataloader):
            y_pre = self.forward(X)
            if (y_pre.argmax().item() == y.item()):
                corrected += 1
            total += 1
        return (corrected / total) * 100

In [ ]:
def evaluate(model, testing_data):
    corrected, total = 0, 0
    testing_dataloader = DataLoader(testing_data, batch_size=1, shuffle=False)

    for batch, (X, y) in enumerate(testing_dataloader):
        y_pre = model.forward(X)
        if (y_pre.argmax().item() == y.item()):
            corrected += 1
        total += 1
    return (corrected / total) * 100

In [ ]:
model = NeuralNetwork()
model.train(training_data, optimizer='adam', epochs=100, lr=1e-3)

In [ ]:
acc_on_trainset = evaluate(model, training_data)
acc_on_testset = evaluate(model, testing_data)
print(f"Accuracy on trainset: {acc_on_trainset}")
print(f"Accuracy on testset: {acc_on_testset}")

* Nhan xet: Model bi overfitting do so lan train qua nhieu (epochs=100)

In [ ]:
model = NeuralNetwork()
model.train(training_data, optimizer='adam', epochs=30, lr=1e-3)

In [ ]:
acc_on_trainset = model.evaluate(training_data)
acc_on_testset = model.evaluate(testing_data)
print(f"Accuracy on trainset: {acc_on_trainset}")
print(f"Accuracy on testset: {acc_on_testset}")